In [ ]:
from sodapy import Socrata
import pandas as pd
import datetime as dt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

modelosSelect = pd.read_csv('ModelosSeleccionados.csv',sep=';')

#Construccion del cliente
client = Socrata("www.datos.gov.co",None)

#ejecución de la consulta
#Nota: Dado el tamaño de la base datos, se requiere un computador con alto poder de procesamiento para trabajar con
#      esa base, por tanto se va a separar la información tomando únicamente los últimos 6 años
diasAtras = 1825
hoy = dt.datetime.now()
hoy = hoy.replace(minute=0,hour=0,second=0,microsecond=0)
anterior = hoy - dt.timedelta(days=diasAtras)
ano = anterior.strftime("%Y")
mes = anterior.strftime("%m")
dia = anterior.strftime("%d")
fechaRef = ano + "-" + mes + "-" + dia + "T00:00:00.000"

results = client.get("gpzw-wmxd",limit=2000000,where='fecha_corte>=\"'+ fechaRef +'\"')  
datosBrutos = pd.DataFrame.from_records(results)
del(results)
#otros archivos a usar
nomFondo = pd.read_csv('NomFondo.txt', sep=';')

In [ ]:
#MAIN -------------------------------
#---------------------
#PARTE 2: Ajustes a los Datos
# 2.1 Organizando la Base de datos
#     Se Limpian algunas columnas innecesarias, se cambian nombres de columna más sencillos para trabajarlos adelante
try:
    datosBrutos = datosBrutos.drop(['nombre_tipo_entidad',
                                    'nombre_entidad',
                                    'tipo_patrimonio',
                                    'nombre_tipo_patrimonio',
                                    'subtipo_patrimonio',
                                    'nombre_subtipo_patrimonio',
                                    'valor_fondo_cierre_dia_t', #valor del fondo en unidades
                                    'rendimientos_abonados_dia', #rendimientos del día 
                                    'precio_cierre_fondo_dia_t', #realmente es el precierrre del dia t
                                    'rentabilidad_30_dias',
                                    'rentabilidad_180_dias',
                                    'rentabilidad_360_dias'], axis=1)
except:
    print('WARNING: Hay columnas que no se encontraron en el ingreso de la base punto2.1')

datosBrutos = datosBrutos.rename(columns={'fecha_corte':'Fecha',
                                          'tipo_entidad':'TipEnt',
                                          'codigo_entidad':'CodEnt',
                                          'codigo_patrimonio':'CodFdo',
                                          'nombre_patrimonio':'NomFdo',
                                          'valor_fondo_cierre_dia_t_1':'VlrCierreAyer',
                                          'valor_unidad_operaciones':'VlrUnd',
                                          'aportes_traslados_recibidos':'MovClient',
                                          'traslados_aportes_valor_pesos':'MovCorp',
                                          'mesada_pensionales_valor':'MovPens',
                                          'retiros_aportes_dif_mesada':'MovDifPens',
                                          'otras_comisiones_valor_pesos':'OtrasComis',
                                          'traslados_aseguradoras_rentas':'MovRentVital',
                                          'otros_retiros_valor_pesos':'MovOtros',
                                          'anulaciones_valor_pesos':'Anulaciones',
                                          'valor_fondo_cierre_dia_t_2':'VlrCierre'})
#    La columna de fecha viene en formato de texto por tanto se transforma
datosBrutos['Fecha'] = datosBrutos['Fecha'].str[:10]
datosBrutos['Fecha'] = pd.to_datetime(datosBrutos['Fecha'])
#   Agunas columnas deben cambiar el tipo de dato para trabajar con ellos más adelante
datosBrutos['CodEnt'] = datosBrutos['CodEnt'].astype('float')
datosBrutos['CodFdo'] = datosBrutos['CodFdo'].astype('float')
datosBrutos['VlrCierre'] = datosBrutos['VlrCierre'].astype('float')
datosBrutos['VlrCierreAyer'] = datosBrutos['VlrCierreAyer'].astype('float')
datosBrutos['MovClient'] = datosBrutos['MovClient'].astype('float')
datosBrutos['MovDifPens'] = datosBrutos['MovDifPens'].astype('float')
datosBrutos['VlrUnd'] = datosBrutos['VlrUnd'].astype('float')
datosBrutos['OtrasComis'] = datosBrutos['OtrasComis'].astype('float')
datosBrutos['MovRentVital'] = datosBrutos['MovRentVital'].astype('float')
datosBrutos['MovOtros'] = datosBrutos['MovOtros'].astype('float')
datosBrutos['Anulaciones'] = datosBrutos['Anulaciones'].astype('float')
datosBrutos['MovCorp'] = datosBrutos['MovCorp'].astype('float')
datosBrutos['MovPens'] = datosBrutos['MovPens'].astype('float')

# 2.2 Agregando algunas columnas que seran necesarias a futuro
#   Se inserta como columna en año, mes, dia, diaSem
datosBrutos['Ano'] = datosBrutos['Fecha'].dt.strftime('%Y')
datosBrutos['Mes'] = datosBrutos['Fecha'].dt.strftime('%b')
datosBrutos['Dia'] = datosBrutos['Fecha'].dt.strftime('%d')
datosBrutos['DiaStr'] = datosBrutos['Fecha'].dt.strftime('%a')

datosBrutos = datosBrutos.merge(nomFondo[['CodEnt',
                                          'FdoTot']],
                                how='left',
                                left_on=['CodEnt','CodFdo'],
                                right_on=['CodEnt','FdoTot'])
#--812.747 Check
datosBrutos = datosBrutos[datosBrutos['FdoTot'].isna()]
#--787.653
datosBrutos.drop('FdoTot', inplace=True, axis=1)

#    puede que en el pasado hayan existido entidades que hoy en dia no, por ello se deben eliminar

#--789.626
datosBrutos = datosBrutos.merge(nomFondo[['CodEnt',
                                          'Nombre']],
                                how='left',
                                on='CodEnt')
#--789.626 Check
datosBrutos = datosBrutos[datosBrutos['Nombre'].notnull()]
datosBrutos.drop('Nombre', inplace=True, axis=1)


#2.4 Identificación de datos Nulos por cada compañia 

#    quitamos duplicados por compañia para saber que días la compañia no subio información
#    se asume que: o Sube todo o no sube nada por fondo
datosUnicos = datosBrutos[['Fecha', 'CodEnt']].drop_duplicates()

#    se cruza para tener el nombre de la empresa
datosUnicos = datosUnicos.merge(nomFondo[['CodEnt','Nombre']],
                                on='CodEnt',
                                how='left')
datosUnicos = pd.pivot_table(datosUnicos,
                             index=['Nombre','CodEnt'],
                             values='Fecha',
                             aggfunc='count').reset_index()
datosUnicos = datosUnicos.rename(columns={'Fecha':'ConteoRg'})
datosUnicos['PctNulos'] = (1 - datosUnicos['ConteoRg'] / diasAtras)
datosUnicos.sort_values('PctNulos', ascending=False, inplace=True)

#3(*) Antes de realizar el proceso descriptivo de los datos se realiza una nueva limpieza a los datos
#     ya que por ejemplo no tiene sentido dejar fondos que ya no existen, o que por ejemplo no han estado 
#     presentes en el periodo de experiencia. Por ello de aqui en adelante se desestiman esos datos 

#     se desestiman aquellas empresas con datos faltantes superior al 1%
datosUnicos.loc[datosUnicos['PctNulos']>0.01,'SeleccCia'] = 'elimina'
datosUnicos.loc[datosUnicos['PctNulos']<0.01,'SeleccCia'] = 'incluye'

datosBrutos = datosBrutos.merge(datosUnicos[['CodEnt','SeleccCia']],
                                on='CodEnt',
                                how='left')
datosBrutos = datosBrutos[datosBrutos['SeleccCia'] == 'incluye']

#     por cada tipo de fondo cuantos datos se tienen
conteoFdos = datosBrutos.pivot_table(index=['CodEnt','CodFdo'],
                                     values='Fecha',
                                     aggfunc='count').reset_index()
conteoFdos = conteoFdos.merge(datosUnicos[['CodEnt','ConteoRg']],
                              on='CodEnt',
                              how='left')

conteoFdos['Decision']= (conteoFdos['Fecha']<conteoFdos['ConteoRg'])
eliminados = int(round(sum(conteoFdos['Decision'])/conteoFdos.shape[0],2)*100)
print('se eliminan el :' + str(eliminados) + '% del nro de Fondos por no tener todo el periodo de experiencia' )

#--739.998
datosBrutos = datosBrutos.merge(conteoFdos[['CodEnt',
                                            'CodFdo',
                                            'Decision']],
                                on=['CodEnt','CodFdo'],
                                how='left')
#--739.998 check
datosBrutos = datosBrutos[datosBrutos['Decision'] != True]
#--336.400 regiustros finales
datosBrutos.drop(['SeleccCia','Decision'], axis=1, inplace=True)

# Se retiran sabados y domingos de la serie
datosBrutos = datosBrutos[datosBrutos['DiaStr'] != 'Sat']
datosBrutos = datosBrutos[datosBrutos['DiaStr'] != 'Sun']

datosBrutos['RetiroNetoPct'] = (datosBrutos['MovClient'] - \
                                datosBrutos['MovCorp'] - \
                                datosBrutos['MovPens'] - \
                                datosBrutos['MovDifPens'] - \
                                datosBrutos['OtrasComis'] - \
                                datosBrutos['MovRentVital'] - \
                                datosBrutos['OtrasComis'])/datosBrutos['VlrCierreAyer']
    
numFdos = datosBrutos.loc[datosBrutos['CodEnt'] == 42, 'CodFdo'].drop_duplicates()


In [ ]:

#3.6 Aproximaxión al modelo construido con un horizonte de tiempo de 1 día

resultados = list()
contador = 0
for fdo in numFdos:
    datosFdo = datosBrutos.loc[((datosBrutos['CodEnt'] == 42) & \
                                (datosBrutos['CodFdo'] == fdo)),
                                ['Fecha','VlrUnd','RetiroNetoPct']]
    subdt = datosFdo.reset_index().iloc[:,3].copy()
    subdt = ((1 + subdt).cumprod()) * 1000
          

    
    valoresDatos = subdt.values
    tamano = int(len(valoresDatos) * 0.8)
    entrenamiento = valoresDatos[:tamano]
    testeo = valoresDatos[tamano:subdt.shape[0]]
    fechasTest = datosFdo.reset_index().loc[tamano:datosFdo.shape[0],'Fecha']

    
    for j in range(3): 
        historiaContinua = list(entrenamiento.copy())
        datosPred = list()
        for i in range(len(testeo)):
            modelo = SARIMAX(historiaContinua, order=(j+1,1,1), enforce_stationarity=False)
            modeloFit = modelo.fit(disp=0)
            salida = modeloFit.forecast()
            estimado = salida[0]
            datosPred.append(estimado)
            
            historiaContinua.append(testeo[i])
        
        prediccionDf = pd.DataFrame(datosPred.copy())
        testeoDf = pd.DataFrame(testeo.copy())
        salida = pd.concat([prediccionDf, testeoDf], axis=1)
        salida.columns = ('prediccion','observado')
        
        if contador == 0:
            resultados = salida[['prediccion','observado']].copy()
            resultados['modelo'] = j + 1
            resultados['fondo'] = fdo
            
            #resultados para calcular falsos negativos
            resultadosRet = salida[['prediccion','observado']].pct_change(1)
            resultadosRet['modelo'] = j + 1
            resultadosRet['fondo'] = fdo
            #resultadosRet['Fecha'] = fechasTest.values
        else:
            pegado = salida[['prediccion','observado']].copy()
            pegado['modelo'] = j + 1
            pegado['fondo'] = fdo
            
            
            #resultados para calcular falsos negativos
            pegadoRet = salida[['prediccion','observado']].pct_change(1)
            pegadoRet['modelo'] = j + 1
            pegadoRet['fondo'] = fdo
            #pegadoRet['Fecha'] = fechasTest.values
            
            
            resultados = pd.concat([resultados,pegado], axis=0)
            resultadosRet = pd.concat([resultadosRet,pegadoRet], axis=0)
        contador += 1
       #--28.68

In [ ]:
#3.7 Calculo del indicador de desempeño
# RMSE
resultados['diff'] = (resultados['prediccion'] - resultados['observado'])**2
desempenoDia = pd.pivot_table(resultados, values = 'diff',
                              index=['modelo','fondo'], 
                              aggfunc=['sum','count']).reset_index()


desempenoDia = desempenoDia.droplevel(1, axis=1)
desempenoDia = desempenoDia.reset_index(drop=True)
desempenoDia['rmse'] = (desempenoDia['sum'] / desempenoDia['count'] ) **0.5

# validacion Falsos Negativos
def falsosNeg(row):
    if row['observado'] < 0 and row['prediccion'] > 0:
        return 1
    else:
        return 0
    
#Falsos Positivos
def falsosPos(row):
    if row['observado'] >= 0 and row['prediccion'] <= 0:
        return 1
    else:
        return 0

def truePos(row):
    if row['observado'] < 0 and row['prediccion'] < 0:
        return 1
    else:
        return 0

def trueNeg(row):
    if row['observado'] >= 0 and row['prediccion'] >= 0:
        return 1
    else:
        return 0

resultadosRet['falsosNeg'] = resultadosRet.apply(lambda x:falsosNeg(x), axis=1)
resultadosRet = resultadosRet.dropna()
desempenoFalsos = pd.pivot_table(resultadosRet[['modelo',
                                                'fondo',
                                                'falsosNeg']],
                                 index=['modelo','fondo'],
                                 values= 'falsosNeg',
                                 aggfunc=['sum','count'])
desempenoFalsos = desempenoFalsos.reset_index()
desempenoFalsos = desempenoFalsos.droplevel(1, axis=1)
desempenoFalsos['falsosNeg'] = desempenoFalsos['sum'] / desempenoFalsos['count']


resultadosRet['falsosPos'] = resultadosRet.apply(lambda x:falsosPos(x), axis=1)

resultadosRet['truePos'] = resultadosRet.apply(lambda x:truePos(x), axis=1)
resultadosRet['trueNeg'] = resultadosRet.apply(lambda x:trueNeg(x), axis=1)

#Validación de diferencia de calculo en estimación vs observado
def cuentaUmbral(row):
    if abs(row['prediccion'] - row['observado']) > 0.025:
        return 1
    else:
        return 0

desempenoUmbral = resultadosRet.copy()
desempenoUmbral['umbral'] = desempenoUmbral.apply(lambda x: cuentaUmbral(x),
                                                  axis=1)
umbral = pd.pivot_table(desempenoUmbral, index=['modelo','fondo'],
                              values='umbral',aggfunc=['sum','count'])
umbral = umbral.reset_index()
umbral = umbral.droplevel(1, axis=1)

umbral['desempenoUmbral'] = umbral['sum'] / umbral['count']


In [ ]:

#*************************************************
# Anexo 1: Graficos para construcción de datos.

datosGraf = datosBrutos.merge(numFdos, on ='CodFdo', how='right')


fig, axs = plt.subplots(4, 2, constrained_layout=True, sharex=True)
x = 0
y = 0
contador = 0
for fdo in numFdos:
    
    subDatosGraf = datosGraf.loc[datosGraf['CodFdo'] == fdo,'RetiroNetoPct']
    
    nombre = list(datosBrutos.loc[datosBrutos['CodFdo']==fdo,'NomFdo'].drop_duplicates())[0]
    axs[x,y].hist(subDatosGraf*100,bins=70, alpha=0.6, label=nombre,
             density=True,color='#E50000')
    if nombre == 'PLAN COMPLEMENTARIO DE PENSIÓN PCP':
        nombre = 'PCP'
    axs[x, y].set_title(nombre)
    contador += 1
    if y == 0 and contador < 4:
        y = 0
    else:
        y = 1
    if contador >= 4 and x == 3:
        del(x)
        x = 0
    else:
        x += 1
    #plt.legend(loc='upper left')
    
    #axs.set_title(nombre)
    # plt.title(nombre)
    # plt.xlabel("Grupo de retiros (%)")
    # plt.ylabel("Frecuencia")
plt.show()
        
# grafico uniendo todos los retiros netos atempoalmente

datosGrafTot = datosBrutos.loc[((datosBrutos['CodEnt'] == 42)),
                                ['Fecha','RetiroNetoPct']]
plt.scatter(datosGrafTot['Fecha'], datosGrafTot['RetiroNetoPct']*100, color='red')
plt.title('Valor retiro Neto')
plt.xlabel('Fecha')
plt.ylabel('Retiro Neto (%)')
plt.show()

#grafico de diferencias
datosDiferencias = resultadosRet.merge(modelosSelect,
                                       on=['fondo','modelo'],
                                       how='right')
datosDiferencias['diff'] = datosDiferencias['prediccion'] - \
    datosDiferencias['observado']
    
plt.hist(datosDiferencias['observado']*100, bins=20, color='red')
plt.ylabel("Frecuencia")
plt.xlabel("Diferencia(%)")
plt.show()

datosDiferencias['observado'].describe()


#Dickey - Fuller TEST

for fdo in numFdos:
    
    subDatosGraf = datosGraf.loc[datosGraf['CodFdo'] == fdo,'RetiroNetoPct']
    nombre = list(datosBrutos.loc[datosBrutos['CodFdo']==fdo,'NomFdo'].drop_duplicates())[0]
    if nombre == 'PLAN COMPLEMENTARIO DE PENSIÓN PCP':
        nombre = 'PCP'
    result = adfuller(subDatosGraf)
    print("----------------------")
    print(nombre)
    print(result[0])
    print(result[1])